# TAX Scraper - Scrapes the data from the PDFs collected by the TAXSpyder

### Imports the libraries

In [48]:
from pyPdf import PdfFileReader
import re
import os
import pandas as pd
import shutil

### Copy O4 Office building files to the main dir

Assumes that the pdf you downloaded are all in a sub dir of the current work dir named "All".  We first copy the ones for O4 Office Buildings into the current work dir.  Some pdf may fail to open.  A failure message will be printed for futher investigation if this happens.

In [49]:
def read_PDF(pdf):
    '''
    Grabs all the text of a pdf
    '''
    pdfOpen = open(pdf, "r")
    pdfRead = PdfFileReader(pdfOpen)
    pdfText = ''
    for page in pdfRead.pages:
        pdfText += page.extractText()
    return pdfText

In [50]:
for path, subdirs, files in os.walk('All'):
    for name in files:
        dst = name
        src = os.path.join(path, name)
        if dst.startswith('NPV'):
            try:
                pdfText = read_PDF(src)
                if pdfText.split('BUILDING CLASS:')[1].split('UNITS:')[0] == '  O4 (Office buildings)':            
                    dst2 = dst.replace('NPV','PTB')
                    src2 = src.replace('NPV','PTB')
                    shutil.copyfile(src, dst)
                    shutil.copyfile(src2, dst2)
            except:
                print 'Failure', name
                pass

### Defines the functions

#### PTB

In [51]:
def read_PTBtaxPDF(pdf):
    '''
    Parses following tax information from a Property Tax Bill (PTB) PDF:
    -previous charges
    -current charges
    -new charges
    -outstanding charges
    '''
    building = pdf[3:-4]
    pdfText = read_PDF(pdf)
    activityThrough = pdfText.split(
        'Quarterly StatementActivity through')[1].split(
        'Mailing address:')[0].split(
        'Statement Billing Summary')[0]
    if pdfText[0:8] == 'Previous':
        previousCharges = pdfText.split(
            'Previous charges$')[1].split('Amount paid$')[0]
        previousCharges = float(re.sub(',', '', previousCharges))
        currentCharges = pdfText.split(
            'Current charges$')[1].split('Total amount due')[0]
        currentCharges = float(re.sub(',', '', currentCharges))
    else:
        previousCharges = 0.00
        currentCharges = 0.00
    if pdfText[0:6] == '001400':
        outstandingCharges = pdfText.split(
            'Outstanding Charges$')[1].split('New Charges$')[0]
        outstandingCharges = float(re.sub(',', '', outstandingCharges))
        newCharges = pdfText.split('New Charges$')[1].split('Amount Due$')[0]
        newCharges = float(re.sub(',', '', newCharges))
    else:
        outstandingCharges = 0.00
        newCharges = 0.00
    return building, activityThrough, previousCharges, currentCharges, outstandingCharges, newCharges

In [52]:
def get_PTBData():
    taxDict = {}
    buildingList = []
    activityThroughList = []
    previousChargesList = []
    currentChargesList = []
    outstandingChargesList = []
    newChargesList = []
    path = os.getcwd()
    for pdf in os.listdir(path):
        if pdf.startswith('PTB'):
            try:
                result = read_PTBtaxPDF(pdf)
                buildingList.append(result[0])
                activityThroughList.append(result[1])
                previousChargesList.append(result[2])
                currentChargesList.append(result[3])
                outstandingChargesList.append(result[4])
                newChargesList.append(result[5])
            except:
                pass
    taxDict['building'] = buildingList
    taxDict['activityThrough'] = activityThroughList
    taxDict['previousCharges'] = previousChargesList
    taxDict['currentCharges'] = currentChargesList
    taxDict['outstandingCharges'] = outstandingChargesList
    taxDict['newCharges'] = newChargesList
    taxDf = pd.DataFrame(taxDict)
    return taxDf

#### NPV

In [53]:
def cats_Start(pdfText):
    '''
    Arranges the categories in order they appear in the pdf
    '''
    try:
        startLast1 = pdfText.index('FactorsUsedByFinance')
    except:
        startLast1 = 99999999
    try:
        startLast2 = pdfText.index('Ifyoubelieve')
    except:
        startLast2 = 99999999
    catStartList = []
    cats = ['NumberofBuildings:',
            'GrossSquareFootage:',
            'NumberofStories:',
            'NumberofResidentialUnits:',
            'StructureType:',
            'GrossResidentialSquareFootage:',
            'Grade:',
            'NumberofCommercialUnits:',
            'ConstructionType:',
            'GrossCommercialSquareFootage:',
            'PrimaryZoning:',
            'YearBuilt:']
    for cat in cats:
        #print cat
        start = pdfText.index(cat)
        catStartList.append(start)
    cats.append('FactorsUsedByFinance')
    catStartList.append(startLast1)
    cats.append('Ifyoubelieve')
    catStartList.append(startLast2)
    return [cat for (st, cat) in sorted(zip(catStartList, cats))][:-1]

In [54]:
def get_NPVIncomeData():
    '''
    Parses all the values for the categories in cats
    from the New Property Value PDFs for for all
    O4 (Office buildings) buildings
    '''
    path = os.getcwd()

    buildingDict = {}
    buildingList = []
    classList = []
    ownerList = []
    estimatedGrossIncomeList = []
    estimatedExpensesList = []
    estimatedNetIncomeList = []
    capitalizationRateList = []
    effectiveTaxRateList = []
    overallCapRateList = []
    dataAsOffList = []

    for pdf in os.listdir(path):
        building = pdf[3:-4]
        if pdf.startswith('NPV'):
            try:
                pdfText = read_PDF(pdf)
                pdfText = pdfText.replace(" ", "")
                pdfText = pdfText.replace("GrossResidentialSq.Footage:", "GrossResidentialSquareFootage:")
                pdfText = pdfText.replace("GrossCommercialSq.Footage:", "GrossCommercialSquareFootage:")

                buildingClass = pdfText.split('BUILDINGCLASS:')[1].split('UNITS:')[0]
                owner = pdfText.split('OWNERNAME')[1].split('PROPERTYADDRESS')[0]
                estimatedGrossIncome = pdfText.split('EstimatedGrossIncome:$')[1].split('EstimatedExpenses:$')[0]
                estimatedExpenses = pdfText.split('EstimatedExpenses:$')[1].split('NetOperatingIncome:Wesubtract')[0]
                estimatedNetIncome = pdfText.split('resultinginanetoperatingincomeof$')[1].split('.BaseCapRate:Weusedacapitalizationrateof')[0]
                capitalizationRate = pdfText.split('.BaseCapRate:Weusedacapitalizationrateof')[1].split('%whichisFinance')[0]
                effectiveTaxRate = pdfText.split('Weaddaneffectivetaxrateof')[1].split('%toaccountfortaxesdue.Addedtogetheryouroverallcapitalizationrateis')[0]
                overallCapRate = pdfText.split('Addedtogetheryouroverallcapitalizationrateis')[1].split('%.TheDepartmentofFinancehas')[0]
                dataAsOff = pdfText.split('TheDepartmentofFinanceestimatesthatasof')[1].split(',theMarketValueforthispropertyis')[0]

                classList.append(buildingClass)
                ownerList.append(owner)
                estimatedGrossIncomeList.append(estimatedGrossIncome)
                estimatedExpensesList.append(estimatedExpenses)
                estimatedNetIncomeList.append(estimatedNetIncome)
                capitalizationRateList.append(capitalizationRate)
                effectiveTaxRateList.append(effectiveTaxRate)
                overallCapRateList.append(overallCapRate)
                dataAsOffList.append(dataAsOff)
                buildingList.append(building)
            except:
                print 'Failure Cats: ', building
                pass
    buildingDict['building'] = buildingList
    buildingDict['buildingClass'] = classList
    buildingDict['owner'] = ownerList
    buildingDict['estimatedGrossIncome'] = estimatedGrossIncomeList
    buildingDict['estimatedExpenses'] = estimatedExpensesList
    buildingDict['estimatedNetIncome'] = estimatedNetIncomeList
    buildingDict['capitalizationRate'] = capitalizationRateList
    buildingDict['effectiveTaxRate'] = effectiveTaxRateList
    buildingDict['overallCapRate'] = overallCapRateList
    buildingDict['dataAsOff'] = dataAsOffList        
    
    NPVIncome_Df = pd.DataFrame(buildingDict)

    return NPVIncome_Df

In [55]:
def get_NPVCatData():
    '''
    Parses all the income and rate values
    from the New Property Value PDFs for for all
    O4 (Office buildings) buildings
    '''
    path = os.getcwd()
    
    buildingList = []
    NumberofBuildingsList = []
    GrossSquareFootageList = []
    NumberofStoriesList = []
    NumberofResidentialUnitsList = []
    StructureTypeList = []
    GrossResSquareFootageList = []
    GradeList = []
    NumberofCommercialUnitsList = []
    ConstructionTypeList = []
    GrossCommSquareFootageList = []
    PrimaryZoningList = []
    YearBuiltList = []

    catDict = {'building': buildingList,
               'NumberofBuildings': NumberofBuildingsList,
               'GrossSquareFootage': GrossSquareFootageList,
               'NumberofStories': NumberofStoriesList,
               'NumberofResidentialUnits': NumberofResidentialUnitsList,
               'StructureType': StructureTypeList,
               'GrossResidentialSquareFootage': GrossResSquareFootageList,
               'Grade': GradeList,
               'NumberofCommercialUnits': NumberofCommercialUnitsList,
               'ConstructionType': ConstructionTypeList,
               'GrossCommercialSquareFootage': GrossCommSquareFootageList,
               'PrimaryZoning': PrimaryZoningList,
               'YearBuilt': YearBuiltList}

    for pdf in os.listdir(path):
        building = pdf[3:-4]
        if pdf.startswith('NPV'):
            try:
                #print building

                pdfText = read_PDF(pdf)

                pdfText = pdfText.replace(" ", "")
                pdfText = pdfText.replace("GrossResidentialSq.Footage:", "GrossResidentialSquareFootage:")
                pdfText = pdfText.replace("GrossCommercialSq.Footage:", "GrossCommercialSquareFootage:")
                pdfText = pdfText.replace("EstimatedBuildingGrossSquareFootage:", "EstimatedBuildingGrossSquareFoot:")

                cats = cats_Start(pdfText)
                catDict['building'].append(building)
                for i in range(12):
                    cat = cats[i][:-1]
                    catValue = pdfText.split(cats[i])[1].split(cats[i+1])[0]
                    catDict[cat].append(catValue)
            except:
                print 'Failure Income: ', building
                pass
            
    NPVCat_Df = pd.DataFrame(catDict)
            
    return NPVCat_Df

### Calls the functions to grab the data and sends it to an Excel book

In [56]:
# Call all the functions
PTBData = get_PTBData()
NPVIncomeData = get_NPVIncomeData()
NPVCatData = get_NPVCatData()

In [57]:
# Merge all the data and send to Excel output
data1 = PTBData.merge(NPVIncomeData, on='building', how='outer')
data2 = data1.merge(NPVCatData, on='building', how='outer')
writer = pd.ExcelWriter('NYC Tax scraping.xlsx')
data2.to_excel(writer, sheet_name='Data')
writer.save()

## Testing

In [58]:
def test(got, expected):
  if got == expected:
    prefix = ' OK '
  else:
    prefix = '  X '
  print '%s got: %s expected: %s' % (prefix, repr(got), repr(expected))

test(data2[data2['building'] == '7 W 48th St']['estimatedNetIncome'].item(),'22,423,353')
test(data2[data2['building'] == '12 W 49th St']['estimatedNetIncome'].item(),'7,967,936')
test(data2[data2['building'] == '25 W 42nd St']['estimatedNetIncome'].item(),'34,804,824')
test(data2[data2['building'] == '59 W 51st St']['NumberofStories'].item(),'43.00')
test(data2[data2['building'] == '109 Park Avenue']['currentCharges'].item(),12109446.68)

 OK  got: u'22,423,353' expected: '22,423,353'
 OK  got: u'7,967,936' expected: '7,967,936'
 OK  got: u'34,804,824' expected: '34,804,824'
 OK  got: u'43.00' expected: '43.00'
 OK  got: 12109446.68 expected: 12109446.68
